In [ ]:
%matplotlib notebook

In [ ]:
import sys
sys.path.append('/home/ggeva/spt3g/spt3g_software/build')
import os
from spt3g import core, std_processing, mapmaker, dfmux, calibration, todfilter, coordinateutils, timestreamflagging, util
from spt3g.coordinateutils.coordsysmodules import FillCoordTransRotations
import numpy as np
import pylab as plt
import matplotlib.image as mgimg
from astropy import units
from astropy.coordinates import SkyCoord

In [ ]:
# Measures the sizes of LR dipoles (in T) for one .g3 file across all three frequency bands.
# Returns total separations (degrees), separation times (seconds),
# RA seprations (degrees), and separation times assuming only scanning in RA (seconds).
def MeasureDipole(file, scanspeed):
    cmins = []
    cmaxs = []
    seps = []
    septimes=[]
    raseps=[]
    raseptimes=[]
    data = [frame for frame in core.G3File(file)]
    freqs = ['90GHz','150GHz','220GHz']
    for freq in range(len(freqs)):
        leftTMapUnweighted = mapmaker.mapmakerutils.remove_weight_t(data[freq]['T'], data[freq]['Wpol'])
        rightTMapUnweighted = mapmaker.mapmakerutils.remove_weight_t(data[3+freq]['T'], data[3+freq]['Wpol'])
        diffTMapUnweighted = leftTMapUnweighted - rightTMapUnweighted

        diffArr=np.asarray(diffTMapUnweighted)
        y = []
        for j in range(len(diffArr)):
            if 299 < j < 400:
                x = []
                for i in range(len(diffArr[j])):
                    if 299 < i < 400:
                        x += [diffArr[j][i]]
                y += [x]
        diffArrZoom = np.asarray(y)

        for j in range(len(diffArrZoom)):
            for i in range(len(diffArrZoom[j])):
                if diffArrZoom[j][i] == diffArrZoom.min():
                    min = [i+330,j+330]
                elif diffArrZoom[j][i] == diffArrZoom.max():
                    max = [i+330,j+330]

        minRa, minDec = diffTMapUnweighted.pixel_to_angle(min[0], min[1])
        maxRa, maxDec = diffTMapUnweighted.pixel_to_angle(max[0], max[1])
        cmin = SkyCoord(ra=minRa*units.radian, dec=minDec*units.radian)
        cmax = SkyCoord(ra=maxRa*units.radian, dec=maxDec*units.radian)
        cmins += [cmin]
        cmaxs += [cmax]
        seps += [cmin.separation(cmax)]
        septimes += [seps[freq].deg/scanspeed]
        raseps += [cmaxs[freq].ra - cmins[freq].ra]
        raseptimes += [raseps[freq].deg/scanspeed]
        
    return seps, septimes, raseps, raseptimes

In [ ]:
# Measure the LR dipoles for coadds of all four (pixelraster) sources
scanspeed = 0.4 # Default scanning speed for pixelraster observations
Sources = ['CenA-pixelraster', 'PMNJ0210-5101-pixelraster', 'MAT5A-pixelraster', 'RCW38-pixelraster']
sources = ['cena-pixelraster', 'pmnj0210-5101-pixelraster', 'mat5a-pixelraster', 'rcw38-pixelraster']
out_root = '/home/ggeva/spt3g/spt3g_software/scratch/ggeva/'
mapsum_root = '/spt/user/ggeva/map_sums/'
job_dir = '/coadd_lr_0.25res_pol/'

seps = []
septimes = []
raseps = []
raseptimes = []
for i in range(len(Sources)):
    fname = mapsum_root + Sources[i] + job_dir + sources[i] + '_coadd_lr_0.25res_sum.g3'
    sep, septime, rasep, raseptime = MeasureDipole(fname, scanspeed)
    seps += [sep]
    septimes += [septime]
    raseps += [rasep]
    raseptimes += [raseptime]

In [ ]:
# Show the measurements for one source
for i in range(len(Sources)):
    print('Source: ', Sources[i])
    print('Separation: ', seps[i])
    print('Separation (time): ', septimes[i])
    print('RA Separation: ', raseps[i])
    print('RA Separation (time): ', raseptimes[i], '\n')